# ASDF Tree Visualization Notebook

In [1]:
# Fill in the path to your data file here
data_path = '../../../data/jw10001001001_01101_00001_mirifushort_s3d_ref.fits'

In [2]:
import asdf
import gwcs

In [3]:
tree = asdf.open(data_path)
tree.tree

/data/dan/stsci/asdf/asdf/extension.py:211: AsdfDeprecationWarning: The module asdf.asdftypes has been deprecated and will be removed in 3.0. Use asdf.types instead. (from gwcs.extension.GWCSExtension)
  AsdfDeprecationWarning)
/data/dan/stsci/asdf/asdf/asdf.py:183: UserWarning: File '../../../data/jw10001001001_01101_00001_mirifushort_s3d_ref.fits' was created with extension 'jwst.transforms.jwextension.JWSTExtension', which is not currently installed (from package jwst-0.13.0a0.dev65+gd1647876)
  warnings.warn(fmt_msg)


{'asdf_library': {'author': 'Space Telescope Science Institute',
  'homepage': 'http://github.com/spacetelescope/asdf',
  'name': 'asdf',
  'version': '2.2.0.dev1530'},
 'history': {'extensions': [<asdf.tags.core.ExtensionMetadata at 0x7fc1c7b85828>,
   <asdf.tags.core.ExtensionMetadata at 0x7fc1c0040940>]},
 'data': <array (unloaded) shape: [4149, 42, 44] dtype: >f4>,
 'dq': <array (unloaded) shape: [4149, 42, 44] dtype: >u4>,
 'err': <array (unloaded) shape: [4149, 42, 44] dtype: >f4>,
 'meta': {'aperture': {'name': '#TODO', 'position_angle': -999.0},
  'asn': {'pool_name': 'singleton', 'table_name': 'singleton'},
  'background': {},
  'bunit_data': 'mJy/arcsec^2',
  'bunit_err': 'mJy/arcsec^2',
  'cal_step': {'assign_wcs': 'COMPLETE',
   'cube_build': 'COMPLETE',
   'dark_sub': 'COMPLETE',
   'dq_init': 'COMPLETE',
   'flat_field': 'COMPLETE',
   'fringe': 'COMPLETE',
   'jump': 'COMPLETE',
   'lastframe': 'COMPLETE',
   'linearity': 'COMPLETE',
   'persistence': 'SKIPPED',
   'phot

## Experiment with using built-in select widget

In [4]:
import ipywidgets as widgets
from IPython.display import display

In [5]:
select = widgets.Select(options=tree.keys())

In [6]:
display(select)

Select(options=('asdf_library', 'history', 'data', 'dq', 'err', 'meta', 'wavedim', 'wavetable', 'weightmap'), …

## Experiment with using built-in accordion widgets

In [7]:
accordion = widgets.Accordion(children=[widgets.Textarea(k) for k in tree.keys()])
for i, key in enumerate(tree.keys()):
    accordion.set_title(i, key)

In [8]:
# This seems to provide a flexible and intuitive layout. We will need to recursively populate the entire tree, though.
display(accordion)

Accordion(children=(Textarea(value='asdf_library'), Textarea(value='history'), Textarea(value='data'), Textare…

### Define widget for displaying WCS objects

In [111]:
from astropy.table import Table, conf


class WCSWidget:
    def __init__(self, wcs):
        self._wcs = wcs
        self._table = self._create_table(wcs)
        
    def _create_table(self, wcs):
        names = ['step index', 'class', 'name', 'inputs', 'outputs', 'size', 'parameters']
        dtypes = ['S' for x in names]
        table = Table(names=names, dtype=dtypes)
        for i, step in enumerate(wcs.forward_transform):
            klass = step.__class__.__name__
            name = step.name
            inputs = step.inputs
            outputs = step.outputs
            size = len(step)
            params = step.parameters            
            table.add_row([str(x) for x in [i, klass, name, inputs, outputs, size, params]])
        return table
        
        
    def show(self):
        return widgets.VBox(children=[
            widgets.Textarea('Expression: {}'.format(self._wcs.forward_transform._format_expression())),
            widgets.Textarea('The WCS pipeline consists of the following steps:'),
            widgets.HTML(self._table._base_repr_(html=True, max_width=-1, show_dtype=False, 
                                                    tableclass=conf.default_notebook_table_class))
        ])

### Recursively construct a representation of a tree

In [112]:
def render_gwcs_pipeline(pipeline):
    return WCSWidget(pipeline).show()

def render_asdf_ndarray(array, label):
    return widgets.Textarea('This is an ASDF NDArray')

def process_asdf_children(node):
    children = []
    for key, item in node.items():
        if isinstance(item, dict):
            children.append(generate_asdf_tree(item))
        elif isinstance(item, gwcs.WCS):
            children.append(render_gwcs_pipeline(item))
        elif isinstance(item, asdf.tags.core.NDArrayType):
            children.append(render_asdf_ndarray(item, key))
        elif isinstance(item, list):
            children.append(widgets.Textarea('{} is a list'.format(key)))
        else:
            children.append(widgets.Textarea('{}'.format(item)))
    return children

def generate_asdf_tree(tree):
    keys = tree.keys()
    accordion = widgets.Accordion(children=process_asdf_children(tree))
    for i, key in enumerate(keys):
        accordion.set_title(i, key)
    return accordion

In [113]:
display(generate_asdf_tree(tree.tree))

Accordion(children=(Accordion(children=(Textarea(value='Space Telescope Science Institute'), Textarea(value='h…